<a href="https://www.kaggle.com/code/lhlong0550/reconstruct-natural-image-from-brain-activity?scriptVersionId=158860178" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

**KAY dataset: Exploration**

In this colab,we are going to explore and understand the data. I encourage you to do your own experiments

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import urllib
import random
import pandas as pd # Library for data analysis and visualization

First download the data:

In [ ]:
fname = "kay_labels.npy"
if not os.path.exists(fname):
  !wget -qO $fname https://osf.io/r638s/download
fname = "kay_labels_val.npy"
if not os.path.exists(fname):
  !wget -qO $fname https://osf.io/yqb3e/download
fname = "kay_images.npz"
if not os.path.exists(fname):
  !wget -qO $fname https://osf.io/ymnjv/download

Then load the data:

In [ ]:
with np.load(fname) as dobj:
    dat = dict(**dobj)

labels = np.load('kay_labels.npy')
val_labels = np.load('kay_labels_val.npy')

Exploring the labels for the training and validation sets. There are 4 rows, the fourth row has the label (as predicted by a DNN on imagenet). Rows 1-3 have contain 3 different levels of the wordnet hierarchy.

**Exercise:** Learn what imagenet and wordnet are.

In [ ]:
print(labels.shape) # 4 rows

print("Number of image: ", labels.shape[1])
print("Number of label for each image: ", labels.shape[0])

print('Example:')
k = 5
print(f'label: {labels[3, k]} which is a/an -> {labels[0, k]} that is a/an -> {labels[1, k]} as a/an -> {labels[2, k]}')

Take a look at some of the labels:

In [ ]:
print(val_labels[:, 6:10])

Learn about the different categories:

In [ ]:
print(np.unique(labels[0]))

How many examples for each category:

In [ ]:
for cat in np.unique(labels[0]):
  print(cat, (labels.T == cat).sum())

**Exploring the data:**

`dat` is a dictionary

In [ ]:
print(dat.keys())

`dat` has the following fields:  
- `stimuli`: stim x i x j array of grayscale stimulus images
- `stimuli_test`: stim x i x j array of grayscale stimulus images in the test set  
- `responses`: stim x voxel array of z-scored BOLD response amplitude
- `responses_test`:  stim x voxel array of z-scored BOLD response amplitude in the test set  
- `roi`: (region of interest) array of voxel labels
- `roi_names`: array of names corresponding to voxel labels

In [ ]:
print(dat["stimuli"].shape) # 1750 images, each 128x128
print(dat["stimuli_test"].shape)
print(dat['responses'].shape)
print(dat['responses'][18])
plt.imshow(dat["stimuli"][300], cmap = 'gray')

The responses are the "activation" of each of the voxels:

In [ ]:
print(dat["responses"].shape) # 8428 voxels in total

ROI's are the part of the brain each voxel corresponds to:

In [ ]:
print(dat['roi'].shape) # each of the voxels has an associated ROI
print(dat['responses'][19][275:325])
print(dat['roi'][275:325])
print(dat['roi_names'][dat['roi'][275:325]]) # ROI has the index of the corresponding region in "roi_names"

In [ ]:
print(np.bincount(dat["roi"])) # How many voxels per region
print(dat["roi_names"]) # all the regions

In [ ]:
v1_responses = dat['responses'][:, dat['roi'] == 1]
v2_responses = dat['responses'][:, dat['roi'] == 2]
v3_responses = dat['responses'][:, dat['roi'] == 3]
v4_responses = dat['responses'][:, dat['roi'] == 6]

# do experiment for each roi
# train them to get the feature vector -> try to generate the image from the feature vector
# from feature vector -> calculate the loss

Let's take a look at some of the images:

In [ ]:
f, axs = plt.subplots(2, 4, figsize=(12, 6), sharex=True, sharey=True)
for ax, im, lbl in zip(axs.flat, dat["stimuli"], labels[-1,:]):
  ax.imshow(im, cmap="gray")
  ax.set_title(lbl)
f.tight_layout()

Each stimulus is associated with a pattern of BOLD response across voxels in visual cortex:

**Exercise:** Read what BOLD is in the context of fMRI. Read the basics of fMRI.

In [ ]:
signal = dat['responses'][200] # Response for stimulis number 8

plt.plot(signal)

plt.xlabel('Voxel index')
plt.ylabel('Response amplitude') # It is given by the z-score of the signal (it's a way of normalization)

plt.figure()
plt.imshow(dat['stimuli'][200], cmap = "gray")

Let's visualize all the responses for all the stimuli:

In [ ]:
f, ax = plt.subplots(figsize=(12, 5))
ax.set(xlabel="Voxel", ylabel="Stimulus")
heatmap = ax.imshow(dat["responses"], aspect="auto", vmin=-1, vmax=1, cmap="bwr") # Shows the
f.colorbar(heatmap, shrink=.5, label="Response amplitude (Z)")
f.tight_layout()

In [ ]:
res1 = dat["responses"][21]

res2 = dat["responses"][51]

similarity = np.dot(res1, res2)

plt.figure()
plt.imshow(dat["stimuli"][21])


plt.figure()
plt.imshow(dat["stimuli"][51])
print(similarity)

And for the validation set:

In [ ]:
f, ax = plt.subplots(figsize=(12, 2.5))
ax.set(xlabel="Voxel", ylabel="Test Stimulus")
heatmap = ax.imshow(dat["responses_test"], aspect="auto", vmin=-1, vmax=1, cmap="bwr")
f.colorbar(heatmap, shrink=.75, label="Response amplitude (Z)")
f.tight_layout()

**Exercises:**
Select 3 images. Make a bar char with the average responses for each ROI for each of the images

In [ ]:
print('responses shape: ', dat['responses'][180].shape)


**Feature extraction Pre-trained models**

First list all available models:

In [ ]:
import torchvision.models as tm
import torch
import os
import urllib
import numpy as np
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from PIL import Image

In [ ]:
# @title Code from previous lab
alexnet = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=True)
alexnet.eval()

# Loading dataset
fname = "kay_labels.npy"
if not os.path.exists(fname):
  !wget -qO $fname https://osf.io/r638s/download
fname = "kay_labels_val.npy"
if not os.path.exists(fname):
  !wget -qO $fname https://osf.io/yqb3e/download
fname = "kay_images.npz"
if not os.path.exists(fname):
  !wget -qO $fname https://osf.io/ymnjv/download

with np.load(fname) as dobj:
    dat = dict(**dobj)

labels = np.load('kay_labels.npy')
val_labels = np.load('kay_labels_val.npy')

# Example image
stimulis = dat['stimuli']
im = dat['stimuli'][18]
# print(im.shape)
plt.imshow(im, cmap="gray")

# Testing the classification
preprocess = transforms.Compose([
            # transforms.ToPILImage(),
            transforms.Resize((224,224)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])
pims = []
print(stimulis.shape)
for i, sti in enumerate(stimulis):
  sti = np.stack([im,im,im],axis=2)*255

  pims.append(Image.fromarray(np.uint8(sti))) # To PIL


#  np.resize(sti, (224, 224))
print(len(pims))

input_tensor = preprocess(pims[0])
input_batch = input_tensor.unsqueeze(0)
print(input_tensor.shape)
with torch.no_grad():
    output = alexnet(input_batch)

probabilities = torch.nn.functional.softmax(output[0], dim=0).detach()
plt.bar(range(len(probabilities)), probabilities)

# Loading classes
!wget https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt

# Printing probable classes
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]

# Show top categories per image
top5_prob, top5_catid = torch.topk(probabilities, 5)
for i in range(top5_prob.size(0)):
    print(categories[top5_catid[i]], top5_prob[i].item())

**Extracting features from a given layer**

In [ ]:
from torchvision.models.feature_extraction import get_graph_node_names
nodes, _ = get_graph_node_names(alexnet)
print(nodes)

Create feature extractor

In [ ]:
from torchvision.models.feature_extraction import create_feature_extractor

feature_extractor_test = create_feature_extractor(
	alexnet, return_nodes=['features.10', 'features.11', 'features.12', 'avgpool', 'flatten', 'classifier.4'])
# `out` will be a dict of Tensors, each representing a feature map
out = feature_extractor_test(torch.zeros_like(input_batch))

Visualizing the feature vector

In [ ]:
print(out['classifier.4'].size())
feat = out['classifier.4'].detach().numpy().squeeze()
plt.bar(range(len(feat)), feat)

How does it look for the example image?

In [ ]:
out = feature_extractor_test(input_batch)
feat = out['classifier.4'].detach().numpy().squeeze()
plt.bar(range(len(feat)), feat)

**Training a decoder from the brain activity**

Setting up the relevant data from the dataset

In [ ]:
roi = 'V4'
idx = dat["roi_names"].tolist().index(roi)
print(dat["roi"] == idx)
responses = dat['responses'][20]
x = responses[dat["roi"] == idx]
print(x.shape)

# Setting up input size
num_classes = len(feat)

**CONFIGURE**

In [ ]:
is_use_collab = False

is_use_alexnet = True
is_use_vgg19 = False

is_mlp_test = False
is_dnn_test = True

is_concatenated_test = False

is_training = True

model = None

In [ ]:
from torchvision.models import vgg19, VGG19_Weights
from torchvision.models.feature_extraction import get_graph_node_names

vgg19 = torch.hub.load('pytorch/vision:v0.10.0', 'vgg19', weights=VGG19_Weights.DEFAULT)
# or any of these variants
# model = torch.hub.load('pytorch/vision:v0.10.0', 'vgg11_bn', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'vgg13', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'vgg13_bn', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'vgg16', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'vgg16_bn', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'vgg19', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'vgg19_bn', pretrained=True)

nodes, _ = get_graph_node_names(vgg19)
print(nodes)


Creating a custom dataset

In [ ]:
dataset_size = dat['stimuli'].shape[0]
test_dataset_size = dat['stimuli_test'].shape[0]

In [ ]:
class CustomVggDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, is_test = False, is_take_all = True, roi_idx = 0):

        if is_test:
          self.size = test_dataset_size
        else:
          self.size = dataset_size

        self.dataset = dataset.copy()
        self.is_take_all = is_take_all
        self.roi_idx = roi_idx
        self.is_test = is_test

    def __getitem__(self, index):

        stimuli = 'stimuli'
        responses = 'responses'

        if self.is_test:
          stimuli+='_test'
          responses+='_test'

        # Select image (stimuli)
        im = self.dataset[stimuli][index].copy()

        preprocess = transforms.Compose([
            # transforms.ToPILImage()
            transforms.Resize((224,224)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])

        nim = np.stack([im,im,im],axis=2)*255
        pim = Image.fromarray(np.uint8(nim)) # To PIL
        input_tensor = preprocess(pim)
        input_batch = input_tensor.clone().unsqueeze(0)

        # Get vgg19 feature vector
        eval_feature_extractor = create_feature_extractor(vgg19, return_nodes=['classifier.3']) # change this for experiment
        out = eval_feature_extractor(input_batch)
        y = out['classifier.3'].detach().squeeze() # change this for experiment - feature vector is a label

        # Select vector of voxels
        responses = self.dataset[responses]
        if self.is_take_all == True:
          x = torch.Tensor(responses[index])
        else:
          x = torch.Tensor(responses[index][self.dataset["roi"] == self.roi_idx])

        return x, y # Data is responses and Label is Feature Vectors extracted from alexnet

    def __len__(self):
        return self.size

    def copy(self):
      return self

In [ ]:
class CustomAlexnetDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, is_test = False, is_take_all = True, roi_idx = 0):

        if is_test:
          self.size = test_dataset_size
        else:
          self.size = dataset_size

        self.dataset = dataset.copy()
        self.is_take_all = is_take_all
        self.roi_idx = roi_idx
        self.is_test = is_test

    def __getitem__(self, index):
        stimuli = 'stimuli'
        responses = 'responses'

        if self.is_test:
          stimuli+='_test'
          responses+='_test'

        # Select image (stimuli)
        im = self.dataset[stimuli][index]

        preprocess = transforms.Compose([
            # transforms.ToPILImage()
            transforms.Resize((224,224)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])

        nim = np.stack([im,im,im],axis=2)*255
        pim = Image.fromarray(np.uint8(nim)) # To PIL
        input_tensor = preprocess(pim)
        input_batch = input_tensor.clone().unsqueeze(0)

        # Get alexnet feature vector
        feature_extractor = create_feature_extractor(alexnet, return_nodes=['classifier.4']) # change this for experiment
        out = feature_extractor(input_batch)
        y = out['classifier.4'].detach().squeeze() # change this for experiment - feature vector is a label

        # Select vector of voxels
        responses = self.dataset[responses]
        if self.is_take_all == True:
          x = torch.Tensor(responses[index])
        else:
          x = torch.Tensor(responses[index][self.dataset["roi"] == self.roi_idx])

        return x, y # Data is responses and Label is Feature Vectors extracted from alexnet

    def __len__(self):
        return self.size

    def copy(self):
      return self


In [ ]:
class CustomConcatDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, is_test = False, is_take_all = True, roi_idx = 0):

        if is_test:
          self.size = test_dataset_size
        else:
          self.size = dataset_size

        self.dataset = dataset.copy()
        self.is_take_all = is_take_all
        self.roi_idx = roi_idx
        self.is_test = is_test

    def __getitem__(self, index):

        stimuli = 'stimuli'
        responses = 'responses'

        if self.is_test:
          stimuli+='_test'
          responses+='_test'

        # Select image (stimuli)
        im = self.dataset[stimuli][index].copy()

        preprocess = transforms.Compose([
            # transforms.ToPILImage()
            transforms.Resize((224,224)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])

        nim = np.stack([im,im,im],axis=2)*255
        pim = Image.fromarray(np.uint8(nim)) # To PIL
        input_tensor = preprocess(pim)
        input_batch = input_tensor.clone().unsqueeze(0)

        # Get vgg19 feature vector
        eval_vgg19_feature_extractor = create_feature_extractor(vgg19, return_nodes=['classifier.3']) # change this for experiment
        out_vgg = eval_vgg19_feature_extractor(input_batch).copy()
        y_vgg19 = out_vgg['classifier.3'].detach().squeeze()

        # Get alexnet feature vector
        alexnet_feature_extractor = create_feature_extractor(alexnet, return_nodes=['classifier.4']) # change this for experiment
        out_alexnet = alexnet_feature_extractor(input_batch).copy()
        y_alexnet = out_alexnet['classifier.4'].detach().squeeze() # change this for experiment - feature vector is a label

        y = np.concatenate((y_alexnet, y_vgg19), axis=0)

        # Select vector of voxels
        responses = self.dataset[responses]
        if self.is_take_all == True:
          x = torch.Tensor(responses[index])
        else:
          x = torch.Tensor(responses[index][self.dataset["roi"] == self.roi_idx])

        return x, y # Data is responses and Label is Feature Vectors extracted from alexnet

    def __len__(self):
        return self.size

    def copy(self):
      return self

Let's train our decoder. Is it doing a good job? (Don't forget to activate the GPU)

*Exercise:* Compute the accuracy for each empoch

In [ ]:
import torch.nn as nn

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


class MLP_Decoder(nn.Module):
    def __init__(self, hidden_size = 4096, num_classes = 4096, input_size = 8428):
        super(MLP_Decoder, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out



In [ ]:
import torch.nn as nn
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class DNN_Decoder(nn.Module):
    def __init__(self, hidden_size, num_classes, input_size = 8428):
        super(DNN_Decoder, self).__init__()
        self.hidden_size = int(input_size/2)
        
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu1 = nn.ReLU()
        
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.relu2 = nn.ReLU()
        
        self.fc3 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        return out


In [474]:
import torch
import torch.nn as nn
from torchviz import make_dot

# Define a simple PyTorch model
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc1 = nn.Linear(10, 5)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(5, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Instantiate the model
model = SimpleModel()

# Create a dummy input (batch size of 1, input size of 10)
dummy_input = torch.randn(1, 10)

# Generate a dynamic computation graph
output = model(dummy_input)
graph = make_dot(output, params=dict(model.named_parameters()))

# Save the graph as a PNG file
graph.render("simple_model", format="png", cleanup=True)

ModuleNotFoundError: No module named 'torchviz'

PREPARE THE DATASET

In [ ]:
custom_dataset = None
custom_test_dataset = None


# You can then use the prebuilt data loader.
if is_use_alexnet:
  custom_dataset = CustomAlexnetDataset(dat)
  custom_test_dataset = CustomAlexnetDataset(dat, is_test = True)
elif is_use_vgg19:
  custom_dataset = CustomVggDataset(dat)
  custom_test_dataset = CustomVggDataset(dat, is_test = True)
elif is_concatenated_test:
    custom_dataset = CustomConcatDataset(dat)
    custom_test_dataset = CustomConcatDataset(dat, is_test = True)

train_loader = torch.utils.data.DataLoader(dataset=custom_dataset.copy(),
                                           batch_size=64,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=custom_test_dataset.copy(),
                                           batch_size=64,
                                           shuffle=True)

Let's train our decoder. Is it doing a good job? (Don't forget to activate the GPU)

*Exercise:* Compute the accuracy for each empoch

In [ ]:
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
from sklearn.metrics import accuracy_score
import numpy as np

if is_training:
  num_epochs = 30

  if is_use_vgg19:
    num_epochs = 20
  # Train the decoder
  learning_rate = 0.001
  hidden_size = 4096

  if is_concatenated_test:
    hidden_size = hidden_size*2
    num_classes = num_classes*2

  if is_mlp_test:
    model = MLP_Decoder(hidden_size, num_classes).to(device)
  elif is_dnn_test:
    model = DNN_Decoder(hidden_size, num_classes, input_size = 8428).to(device)

  # define loss function - optimization - lr scheduler
  criterion = nn.MSELoss()
  optimizer = torch.optim.Adam(model.parameters(), weight_decay=0.01, lr=learning_rate)
  # using ReduceLROnPlateau to reduce lr when there's no improvement with loss or accuracy
  scheduler = ReduceLROnPlateau(optimizer, patience = 3, factor = 0.1, min_lr=0.000001)

  total_step = len(train_loader)

  for epoch in range(num_epochs):
      total_loss = 0.0
      print("Running epoch {} with learning rate {}".format(epoch+1, optimizer.param_groups[0]['lr']))
      for i, (voxels, afeatures) in enumerate(train_loader):
          optimizer.zero_grad()
          # Move tensors to the configured device
          voxels = voxels.to(device)
          afeatures = afeatures.to(device)

          # Forward pass
          outputs = model(voxels)
          loss = criterion(outputs, afeatures)

          # Backward and optimize
          loss.backward()
          optimizer.step()
          total_loss += loss.item()

          if (i+1) % 10 == 0 or (i+1) % total_step == 0:
              print ('Step [{}/{}], Loss: {:.4f}'
                    .format(i+1, total_step, loss.item()))
      average_loss = total_loss / len(train_loader)
      validation_loss = average_loss
      scheduler.step(validation_loss)

      print(f'Epoch [{epoch + 1}/{num_epochs}], Current Loss: {average_loss:.4f}, Learning Rate: {optimizer.param_groups[0]["lr"]:.4f}')


In [ ]:
import numpy as np

# Example integer arrays
array1 = np.array([1, 2, 3])
array2 = np.array([4, 5, 6])

# Concatenate arrays along a specified axis
result = np.concatenate((array1, array2), axis=0)

print(result)

In [ ]:

import pickle

assessment_path = ''

if is_use_collab:
    from google.colab import drive
    drive.mount('/content/drive')

    assessment_path = '/content/drive/MyDrive/DEEP NEURAL NETWORKS AND LEARNING SYSTEMS/ASSESSMENT/'

model_path = f'{assessment_path}'

if is_training:
    if is_use_alexnet:
        model_path+='alexnet'
    elif is_use_vgg19:
        model_path+='vgg19'
    elif is_concatenated_test:
        model_path+='concatenated'
    
    if is_mlp_test:
        model_path+='_mlp_decoder.pkl'
    elif is_dnn_test:
        model_path+='_dnn_decoder.pkl'
        
    with open(model_path,'wb') as handle:
        pickle.dump(model, handle)

  #cosine similarities

In [ ]:
def predict_with_model(test_dl, model):
  y_preds, y_tests = list(), list()
  model.eval()
  test_loss = 0
  with torch.no_grad():
    for i, (input, targets) in enumerate(test_dl):
      # Evaluate the model on the test set
      input = input.to(device)
      targets = targets.to(device)
      y_hat = model(input)
      test_loss = criterion(y_hat, targets).item()
      # Retrieve numpy array
      y_hat = np.array(y_hat.detach().cpu())
      y_test = np.array(targets.cpu())
      # y_test = y_test.reshape((len(y_test), 1))
      print('y_hat: ', y_hat.shape)
      print('y_test: ', y_test.shape)
      # Round to class values
      y_hat = y_hat.round()

      # Store
      y_preds.append(y_hat)
      y_tests.append(y_test)
    test_loss /= len(test_dl)
    y_preds, y_tests = np.vstack(y_preds), np.vstack(y_tests)
    print('\nTest set: Average loss: {:.4f}'.format(test_loss))
    return y_preds, y_tests, test_loss

In [ ]:
import pickle
assessment_path = ''

if is_use_collab:
    from google.colab import drive
    drive.mount('/content/drive')
    import pickle
    assessment_path = '/content/drive/MyDrive/DEEP NEURAL NETWORKS AND LEARNING SYSTEMS/ASSESSMENT/'

model_path = f'{assessment_path}'

if is_use_alexnet:
  model_path+='alexnet'
elif is_use_vgg19:
  model_path+='vgg19'
elif is_concatenated_test:
    model_path+='concatenated'

if is_mlp_test:
  model_path+='_mlp_decoder.pkl'
elif is_dnn_test:
  model_path+='_dnn_decoder.pkl'
print(model_path)
with open(model_path,'rb') as file:
  load_model = pickle.load(file)

predictions, actuals, test_loss = predict_with_model(test_loader, load_model)

# use othe

In [ ]:
print(predictions.shape)
print(actuals.shape)

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from sklearn.metrics.pairwise import cosine_similarity

print("pred_feat: ", predictions.shape)
print("pretrained_feat: ", actuals.shape)

cosine_preds = predictions.copy()
cosine_actuals = actuals.copy()

similarity = []

for i in range(predictions.shape[0]):
  predictions[i] = predictions[i]/np.linalg.norm(predictions[i])
  actuals[i] = actuals[i]/np.linalg.norm(actuals[i])
dot_product = np.dot(predictions, actuals.T)


colors = [(0, 0, 0), (0, 0, 1)]  # (black, red)
n_bins = 100  # Number of bins
cmap_name = "black_blue"
custom_cmap = LinearSegmentedColormap.from_list(cmap_name, colors, N=n_bins)
# Plot the matrix as a heatmap
plt.imshow(dot_product, cmap=custom_cmap, interpolation='nearest')

plt.xlabel('Predicted feature vectors')
plt.ylabel('Actual feature vectors')
# Add a colorbar to the right of the plot
plt.colorbar()

# Show the plot
plt.show()

Let's train our decoder. Is it doing a good job? (Don't forget to activate the GPU)

*Exercise:* Compute the accuracy for each empoch

In [ ]:
# First select an image from the validation

# Compute the features

# Compute the feature vector from alexnet for the same image

# COmpute the dot product

# Create matrix comparing at least 20 images

**Average feature vector for each category**
On the validation set, take the features extracted from alexnet and compute an average feature set for each category

In [ ]:
# Your code here

**Decode activity**
Take the ROI voxel activations for each image in the validation set and decode the corresponding feature vector

In [ ]:
# your code here
# predict responses to get feature vector

**Compare the averae with the decoded features** Iterate over all averages and compare the two vectors, average and decode. The best match is the corresponding category.

In [ ]:
# Your code here
# get result from 2 cells above and compare with dot product